#Imports and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import accuracy_score
import sklearn.decomposition
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from time import time

In [2]:
fashion_mnist = keras.datasets.fashion_mnist;
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data();
"""
0	T-shirt/top
1	Trouser
2	Pullover
3	Dress
4	Coat
5	Sandal
6	Shirt
7	Sneaker
8	Bag
9	Ankle boot
"""

label_names=['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal',
             'Shirt','Sneaker','Bag','Ankle boot']

In [3]:
X_train = np.zeros([60000,784]) 
for i in range(60000): 
    img=train_images[i,:,:]
    X_train[i,:] = img.reshape([784])

X_test = np.zeros([10000,784]) 
for i in range(10000):
    img=test_images[i,:,:]
    X_test[i,:] = img.reshape([784])

X_sub = X_train[:1000,:]
sub_labels = train_labels[:1000]

#1

In [4]:
col_means = np.mean(X_train, axis = 0)
X_tilda = X_train - col_means
X_test_centered = X_test - col_means

k=190
PCA = sklearn.decomposition.PCA(n_components = k)
PCA.fit(X_tilda)
Y_train = PCA.transform(X_tilda)
Y_test = PCA.transform(X_test_centered)

In [ ]:
start1 = time()
#fits models
for i in range(10):
    for j in range(i+1,10):
        globals()['ovo'+str(i)+str(j)] = LogisticRegression(penalty='none')
        globals()['ovo'+str(i)+str(j)].fit(Y_train[(train_labels==i) | (train_labels==j),], 
                                           train_labels[(train_labels==i) | (train_labels==j)]);

#predicts
pred_mat = np.zeros([Y_test.shape[0],45])
col = 0
for i in range(10):
    for j in range(i+1,10):
        
        pred_mat[:,col] = globals()['ovo'+str(i)+str(j)].predict(Y_test)
        col+=1
pred_mat = pred_mat.astype('int')

#print(pred_mat[0,:])
preds1=np.zeros([pred_mat.shape[0]])
for i in range(pred_mat.shape[0]):
    
    preds1[i] = np.argmax(np.bincount(pred_mat[i,:]))

score1 = accuracy_score(y_true = test_labels, y_pred = preds1)
score1
end1 = time()

In [ ]:
#1v1, 1 vs rest, multinomial
start2 = time()
logreg2 = LogisticRegression(penalty='none', multi_class = 'ovr',max_iter = 500)
logreg2.fit(Y_train, train_labels)
preds2 = logreg2.predict(Y_test)
score2 = accuracy_score(y_true = test_labels, y_pred = preds2)
end2 = time()

start3=time()
logreg3 = LogisticRegression(penalty='none', multi_class = 'multinomial',max_iter=500).fit(Y_train, train_labels)
preds3 = logreg3.predict(Y_test)
score3 = accuracy_score(y_true = test_labels, y_pred = preds3)
end3 = time()

In [9]:
start4 = time()
lda = LinearDiscriminantAnalysis()
lda.fit(Y_train,train_labels)
preds4 = lda.predict(Y_test)
score4=accuracy_score(y_true=test_labels, y_pred=preds4)
end4 = time()

In [10]:
print(score1)
print(score2)
print(score3)
print(score4)
print(end1-start1)
print(end2-start2)
print(end3-start3)
print(end4-start4)

0.8441
0.841
0.8393
0.8087
23.370010375976562
161.95142078399658
65.16100525856018
3.533728837966919
